#파일 다운 압축해제


##8/30 stand - train good (1100) + sit - test (360) = 1460

##구글 드라이브 마운트

In [ ]:
from google.colab import drive
drive.mount('/gdrive')

In [ ]:
# t= !find / -name "SIT.zip"
# print(a)

In [ ]:
cd /gdrive/MyDrive/PROJECT/

In [ ]:
cd ./DATA/SIT1500/test/BAD3

In [ ]:
!pwd
!ls -al

In [ ]:
#앞의 cd 명령어 수행해야 함
import os
a= os.getcwd()#현재 위치 주소
print(type(a))
print(a)

import string
b=str(a)+'/*.jpg'
#print(a,'/*.jpg'.rstrip(" "))
print(b)

c=str(a)+'/'
print(c)

##about SITu

In [ ]:
!ls
!cd SITu

In [ ]:
!ls -a

#이미지 전처리 (크기 감량 위함)

##이미지 로딩

In [ ]:
!pwd

In [ ]:
#폴더의 모든 사진을 불러옴 - 확인용 코드(안돌려도 됨)
import glob #glob 모듈의 glob 함수는 사용자가 제시한 조건에 맞는 파일명을 리스트 형식으로 반환한다.

for filename in glob.glob(b):
  print(filename)
  # name_list = filename.split("/")
  # print(name_list)

##YOLO 인식

In [ ]:
cd /content/

In [ ]:
!pwd

In [ ]:
!git clone https://github.com/dhrim/darkeras-yolov4

In [ ]:
%cd darkeras-yolov4

In [ ]:
!wget -O weights/yolov3.weights https://pjreddie.com/media/files/yolov3.weights

In [ ]:
!pwd

##모델 로딩

In [ ]:
import yolov3_wrapper
model = yolov3_wrapper.YoloV3Wrapper("weights/yolov3.weights")
model.save("yolov3.h5")#

###YOLO 인식 / 이미지 자르기
- 지금은 stand에 대해서만 실행한다
- mv 이동대상 이동할 디렉터리

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

import glob
import os

import core
import colorsys
import random


def draw_bbox(file_name1, bboxes, show_label=True):
    """
    bboxes: [x_min, y_min, x_max, y_max, probability, cls_id] format coordinates.
    """
    Msc=[0]
    #classes : 딕셔너리 가능 항목 가져옴 num:str
    try:
        classes = class_names
    except NameError:
        classes = core.utils.read_class_names("./data/classes/coco.names")
    num_classes = len(classes)
    image_h, image_w, _ = file_name1.shape#이미지 크기

    hsv_tuples = [(1.0 * x / num_classes, 1., 1.) for x in range(num_classes)]
    colors = list(map(lambda x: colorsys.hsv_to_rgb(*x), hsv_tuples))
    colors = list(map(lambda x: (int(x[0] * 255), int(x[1] * 255), int(x[2] * 255)), colors))

    random.seed(0)
    random.shuffle(colors)
    random.seed(None)

    for i, bbox in enumerate(bboxes):#indexing(count, list v)
        coor = np.array(bbox[:4], dtype=np.int32)
        
        fontScale = 0.5
        score = bbox[4]
        Msc=np.append(Msc,score)
        class_ind = int(bbox[5])

        if classes[class_ind] in "person" :
          if score>=np.max(Msc):
            bbox_color = colors[class_ind]
            bbox_thick = int(0.6 * (image_h + image_w) / 600)
            c1, c2 = (coor[0], coor[1]), (coor[2], coor[3]) 
            cv2.rectangle(file_name1, c1, c2, bbox_color, bbox_thick)
            score=np.max(Msc)
            print('%s'%classes[class_ind],"SC",score)
            bbox_mess = '%s: %.2f' % (classes[class_ind], score)#출력되는 text
            t_size = cv2.getTextSize(bbox_mess, 0, fontScale, thickness=bbox_thick//2)[0]
            cv2.rectangle(file_name1, c1, (c1[0] + t_size[0], c1[1] - t_size[1] - 3), bbox_color, -1)  # filled#labeling2
            #이미지S 파일,시작점(x,y),종료점(x,y)

            c1_1= (c1[0] + t_size[0], c1[1] - t_size[1] - 3)
            #이미지 추출은 y:y+h, x:x+w로 주어준다.

            ####PP=file_name1[c1[1]:c1_1[1],c1[0]:c1_1[0]]#+x값>,-y값(위 아래)<#y맞음
            #print("x/y",c1_1[1]-c1[1],"/",c1[0]-c1_1[0])#높이, 너비 -
            PP=file_name1[c1[1]:c2[1],c1[0]:c2[0]]

            PP = cv2.cvtColor(PP, cv2.COLOR_RGB2BGR) # 색상 변경
            cv2.imwrite("/gdrive/MyDrive/PROJECT/DATA0829/SIT1500/TEST/BAD3/" + name_list[len(name_list) - 1], PP)# 사람 부분만 이미지 추출 #자기의 tmp 폴더에 good 폴더 미리 생성
            ####print(c1[0],c1_1[0],"//",c1[1],c1_1[1])#376 476 // 0 -13
            
            cv2.putText(file_name1, bbox_mess, (c1[0], c1[1]-2), cv2.FONT_HERSHEY_SIMPLEX,fontScale, (0, 0, 0), bbox_thick//2, lineType=cv2.LINE_AA)#labeling2
    #print("Msc",Msc,"//",max(Msc)) #Msc [0.98066556 0.9436748  0.41189495 0.26212615] // 0.9806655645370483

    return file_name1

In [ ]:
for filename in glob.glob(b):  # "현 디렉터리"의 모든 사진들을 불러옴
  print(filename)
  name_list = filename.split("/")
  new_filename = c + name_list[len(name_list) - 1] # 사진 파일 이름을 new_filename에 저장
  INPUT_SIZE = 800

  image_path = filename
  file_name = cv2.imread(image_path)#이미지 파일 경로
  file_name = cv2.cvtColor(file_name, cv2.COLOR_BGR2RGB)#입력 이미지,색상 변환 코드

  plt.figure(figsize=(12,20))
  plt.imshow(file_name)
  plt.show()

  #2
  bboxes = model.predict(file_name)#keras코드에서 boundary box로 bounding 됨

  #3
  dummy = np.copy(file_name)

  for box in bboxes:
    x1, y1, x2, y2 = int(box[0]), int(box[1]), int(box[2]), int(box[3])
    conf = box[4]
    cls_id = int(box[5])
    dummy = cv2.rectangle(dummy, (x1,y1), (x2,y2), (255,0,0), 2)#빨간색, 선의 두께는 2

  plt.figure(figsize=(12,20))#(그래프 가로, 세로)
  #plt.gca().set_title("yolov3") # get current figure 현재의 figure를 확인하기 위한 방법 /입력한 제목 추가 
  #Axes반환한다(실제 이미지데이터의 역할)
  # plt.imshow(dummy)#원하는 사이즈의 픽셀을 원하는 색으로 채워서 만든 그림 
  # plt.show()
  #4
  file_name = draw_bbox(file_name, bboxes)
  #plt.figure(figsize=(12,20))#(x축의 길이,y축의길이)
  #plt.imshow(file_name)
  #plt.show()
print("\nALL DONE")
  #cv2.imwrite('/gdrive/MyDrive/tmp/new/' + name_list[len(name_list) - 1], file_name)
  #사람만 인식된 상태로 저장 (해당 부분만 크롭은 안된 상태 - 이는 함수내에서 실행됨)
  

In [ ]:
#코드 셀 실행 결과 삭제 코드
from google.colab import output
output.clear()

# CNN Learning

In [ ]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

from tensorflow import keras
from tensorflow.keras import optimizers
from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow.keras import optimizers
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dropout, Flatten, Dense, BatchNormalization, Conv2D, MaxPooling2D, Input
from tensorflow.keras import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.callbacks import Callback

import time

from tensorflow.keras.applications import efficientnet
from tensorflow.keras.applications import EfficientNetB2
import tensorflow as tf

from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot

model = keras.Sequential()
model.add(Input((224,224,3)))
model.add(Conv2D(64, (3,3), padding='same'))
model.add(Conv2D(64, (3,3), padding='same'))
model.add(MaxPooling2D())
model.add(Conv2D(128, (3,3), padding='same'))
model.add(Conv2D(128, (3,3), padding='same'))
model.add(MaxPooling2D())
model.add(Conv2D(256, (3,3), padding='same'))
model.add(Conv2D(256, (3,3), padding='same'))
model.add(Conv2D(256, (3,3), padding='same'))
model.add(MaxPooling2D())
model.add(Conv2D(512, (3,3), padding='same'))
model.add(Conv2D(512, (3,3), padding='same'))
model.add(Conv2D(512, (3,3), padding='same'))
model.add(MaxPooling2D())
model.add(Conv2D(512, (3,3), padding='same'))
model.add(Conv2D(512, (3,3), padding='same'))
model.add(Conv2D(512, (3,3), padding='same'))
model.add(MaxPooling2D())
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(BatchNormalization())
model.add(Dense(5, activation='softmax'))

#SVG(model_to_dot(model, show_shapes=True).create(prog='dot', format='svg'))#구조 이미지화

In [ ]:
def normalize(image):
  return image/255.0

preprocessor = normalize
BATCH_SIZE = 32#default value
#BATCH_SIZE = 128

train_data_generator = ImageDataGenerator(
      # rotation_range=10, #bad1~4,good 5개
      # width_shift_range=0.1,
      # height_shift_range=0.1,
      horizontal_flip=True,
      vertical_flip=False,
      preprocessing_function=preprocessor
).flow_from_directory(
      #directory="/gdrive/MyDrive/PROJECT/DATA0829/STAND1500/TRAIN/BAD1/",#학습시킬 경로
      "/gdrive/MyDrive/PROJECT/DATA0829/STAND1500/TRAIN/",#rs 5class
      #classes=['BAD1'],
      target_size=(224,224),
      batch_size=BATCH_SIZE,
      class_mode='sparse'#분류 방식
)

test_data_generator = ImageDataGenerator(
      preprocessing_function=preprocessor
).flow_from_directory(
      #directory="/gdrive/MyDrive/PROJECT/DATA0829/SIT1500/TEST/BAD1/", #test경로
      "/gdrive/MyDrive/PROJECT/DATA0829/SIT1500/TEST",#rs 4class
      #classes=['BAD1'],
      target_size=(224,224),
      batch_size=BATCH_SIZE,
      class_mode='sparse'
)

- class_mode : 분류 방식에 대해서 지정합니다.
1. categorical : 2D one-hot 부호화된 라벨이 
반환됩니다.
2. binary : 1D 이진 라벨이 반환됩니다.
3. sparse : 1D 정수 라벨이 반환됩니다.
4. None : 라벨이 반환되지 않습니다.

### 확인용 코드

In [ ]:
print(train_data_generator)

##모델 체크 포인트

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint
model_check_point = ModelCheckpoint(
    'CUTDATA.h5', 
    monitor='val_loss', 
    verbose=1, s
    save_best_only=True)
#모델체크포인트

##Result

In [ ]:
# model.compile(loss='sparse_categorical_crossentropy', optimizer="RMSprop", metrics=['acc'])
# #good,bad만 분류하면되니까 2진분류


# loss, acc = model.evaluate(
#       stand_test_data,
#       steps=stand_test_data.samples/BATCH_SIZE
# )
# test_x, test_y = next(iter(stand_test_data))
# y_ = model.predict(test_x)
# predicted = np.argmax(y_, axis=-1)

model.summary()

- error (08/30)
Found 0 images belonging to 0 classes.


sv : https://datascience.stackexchange.com/questions/51671/keras-flow-from-directory-returns-0-images


https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/image/ImageDataGenerator

- about loss choosing
https://medium.com/deep-learning-with-keras/which-activation-loss-functions-in-multi-class-clasification-4cd599e4e61f

In [ ]:
#컴파일
model.compile(loss='sparse_categorical_crossentropy', optimizer="RMSprop", metrics=['acc'])
#good,bad만 분류하면되니까 2진분류이긴 한데 지금은 아니다.
#model.compile(loss='binary_crossentropy', optimizer="RMSprop", metrics=['acc'])
#학습
history = model.fit(
      train_data_generator,
      validation_data=train_data_generator,
      steps_per_epoch=train_data_generator.samples/BATCH_SIZE,
      validation_steps=train_data_generator.samples/BATCH_SIZE,      
      epochs=100, callbacks=[model_check_point])
#평가  
#loss,val_loss=model.evaluate()
#예측  model.predict()

plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.show()